In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv('state_consumption.csv')


# Split data into features and target
X = data[['Goa', 'Gujarat', 'Maharashtra', 'DNH', 'Rajasthan']]
data['total_electricity_consumption'] = data['Goa'] + data['Gujarat'] + \
    data['Maharashtra'] + data['DNH'] + data['Rajasthan']
y = data['total_electricity_consumption']

data
# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the data for LSTM input (samples, time steps, features)
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y, test_size=0.2, random_state=42)

model_filename = "my_table_model_lstm.h5"
if os.path.exists(model_filename):
    # Load the existing LSTM model
    model = tf.keras.models.load_model(model_filename)
    print("Existing LSTM model loaded.")
else:
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, activation='relu', input_shape=(1, 5)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # No activation function for regression
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    print("New LSTM model created.")

# Check if model needs retraining
if not os.path.exists(model_filename):
    # Train the new LSTM model
    model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test))
else:
    # Retrain the existing LSTM model
    model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test))

model.save(model_filename)
print("LSTM Model saved.")

# Use the model to predict electricity consumption for the last 5 rows
X_test_last_5 = X_reshaped[-5:]
y_true_last_5 = y.tail(5)

# Predict using the LSTM model
y_pred_last_5 = model.predict(X_test_last_5)

for i in range(5):
    print(f"True Consumption: {y_true_last_5.iloc[i]} kWh")
    print(f"Predicted Consumption: {y_pred_last_5[i][0]} kWh")
    print()

# Evaluate the model
mae_lstm = mean_absolute_error(y_true_last_5, y_pred_last_5)
mse_lstm = mean_squared_error(y_true_last_5, y_pred_last_5)
rmse_lstm = np.sqrt(mse_lstm)

print(f"LSTM Mean Absolute Error (MAE): {mae_lstm:.2f}")
print(f"LSTM Mean Squared Error (MSE): {mse_lstm:.2f}")
print(f"LSTM Root Mean Squared Error (RMSE): {rmse_lstm:.2f}")

Existing LSTM model loaded.
Epoch 1/500
12/12 [==============================] - 1s 20ms/step - loss: 958.6312 - mae: 16.8803 - val_loss: 266.6990 - val_mae: 10.6927
Epoch 2/500
12/12 [==============================] - 0s 5ms/step - loss: 924.6095 - mae: 16.5597 - val_loss: 254.7031 - val_mae: 10.3848
Epoch 3/500
12/12 [==============================] - 0s 6ms/step - loss: 892.3748 - mae: 16.2395 - val_loss: 246.7484 - val_mae: 10.3157
Epoch 4/500
12/12 [==============================] - 0s 5ms/step - loss: 861.1891 - mae: 15.9160 - val_loss: 235.6721 - val_mae: 9.9675
Epoch 5/500
12/12 [==============================] - 0s 4ms/step - loss: 832.5576 - mae: 15.7168 - val_loss: 231.2401 - val_mae: 10.1069
Epoch 6/500
12/12 [==============================] - 0s 5ms/step - loss: 802.6299 - mae: 15.3943 - val_loss: 221.4652 - val_mae: 9.7873
Epoch 7/500
12/12 [==============================] - 0s 5ms/step - loss: 776.9778 - mae: 15.0663 - val_loss: 212.4426 - val_mae: 9.4686
Epoch 8/500
12/

c:\Users\rohit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 123ms/step
True Consumption: 983.9000000000001 kWh
Predicted Consumption: 984.1852416992188 kWh

True Consumption: 888.3 kWh
Predicted Consumption: 891.7362670898438 kWh

True Consumption: 933.1 kWh
Predicted Consumption: 937.5568237304688 kWh

True Consumption: 945.3000000000001 kWh
Predicted Consumption: 950.8965454101562 kWh

True Consumption: 942.7 kWh
Predicted Consumption: 945.9341430664062 kWh

LSTM Mean Absolute Error (MAE): 3.40
LSTM Mean Squared Error (MSE): 14.71
LSTM Root Mean Squared Error (RMSE): 3.83
